In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

In [7]:
pred_cc_img_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc' 
#/image_102_pred_cc_binary.tif'
lbl_cc_img_path = '/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc' #
#/image_102_cc_binary.tif'

### Common Function

In [8]:
range_found = 'no range found'
def find_iou_range(iou_score):
    if 0 <= iou_score <= 20:
        range_found = ' <= 20%'
    elif 20 <= iou_score <= 30:
        range_found = '20+ to 30%'
    elif 30 <= iou_score <= 40:
        range_found = '30+ to 40%'
    elif 40 <= iou_score <= 50:
        range_found = '40+ to 50%'
    elif 50 <= iou_score <= 60:
        range_found = '50+ to 60%'
    elif 60 <= iou_score <= 70:
        range_found = '60+ to 70%'
    elif 70 <= iou_score <= 80:
        range_found = '70+ to 80%'
    elif 80 <= iou_score <= 90:
        range_found = '80+ to 90%'        
    elif 90 <= iou_score <= 95:
        range_found = '90+ to 95%'        
    elif 95 <= iou_score <= 100:
        range_found = '95+ to 100%'                

    return range_found

In [9]:
def find_iou(gt_bw_img, pred_bw_img):
    if(gt_bw_img.shape == pred_bw_img.shape):
        intersection = np.logical_and(gt_bw_img, pred_bw_img)
        union = np.logical_or(gt_bw_img, pred_bw_img)
        iou_score = np.sum(intersection) / np.sum(union)
        return iou_score
    else:
        raise Exception('***** find_iou() fails.')

### Read Images

In [11]:
lst_pred_cc_imgs = [img for img in os.listdir(pred_cc_img_path) if os.path.isfile(os.path.join(pred_cc_img_path, img))]
lst_pred_cc_imgs[:2], len(lst_pred_cc_imgs)

(['image_48_cc.png', 'image_24_cc.png'], 626)

### Find IOU and Write it to CSV

In [13]:
lst_final_csv_output= []
for idx, pred_img_name in enumerate(lst_pred_cc_imgs, start=1):
    try:
#         print(pred_img_name)
        pred_img_path_with_filename = os.path.join(pred_cc_img_path, pred_img_name)

        lbl_img_name = pred_img_name.replace('.png', '.tif')
        lbl_img_path_with_filename = os.path.join(lbl_cc_img_path, lbl_img_name)
        print(lbl_img_path_with_filename)
        print(pred_img_path_with_filename)

        lbl_bw_img = cv.imread(lbl_img_path_with_filename, cv.IMREAD_GRAYSCALE)
        pred_bw_img = cv.imread(pred_img_path_with_filename, cv.IMREAD_GRAYSCALE)

        #find IOU score
        iou_score_original = find_iou(lbl_bw_img, pred_bw_img)
        iou_score = '{0:.4f}'.format(iou_score_original)

        iou_perc = '{0:.4f}'.format(iou_score_original*100)
    #         print(type(iou_perc), iou_perc)
        iou_range = find_iou_range(float(iou_perc))
        img_name = pred_img_name.replace('pred_cc_binary', '')
        print_str = '# {} :\t {} \t {} \t {} \t {} \t {}'.format(idx, lbl_img_name, pred_img_name, iou_score, iou_perc, iou_range)
        print(print_str)
    #         print('Processing Image # %d : %s \t %s \t %s \t %s' % (idx, img_name, pred_file_name, iou_score, iou_perc))
    #     write_current_progress_status(print_str)
        #construct LIST to store it in csv file
        lst_final_csv_output.append([idx, img_name, iou_score , iou_perc, iou_range])
    except Exception as exception:
        print('-' * 50)
        print('Exception raised %s for the Image %s' % (str(exception), img_name))
        print('-' * 50)
#         write_log(str(exception), img_name )
    finally:
        print('~' * 100)

        
#store the csv output
df = pd.DataFrame(lst_final_csv_output, columns=['#', 'image_name', 'IOU_Score', 'IOU_Perc', 'Class'])
df.to_csv('iou_score.csv', index=False)
print('total files processed : ', idx)
print('csv generated successfully')    

/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_48_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_48_cc.png
# 1 :	 image_48_cc.tif 	 image_48_cc.png 	 0.9666 	 96.6597 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_24_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_24_cc.png
# 2 :	 image_24_cc.tif 	 image_24_cc.png 	 0.9706 	 97.0617 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_329_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_329_cc.png
# 3 :	 image_329_cc.tif 	 image_329_cc.png 	 0.8862 	 88.6162 	 80+ to 9

# 84 :	 image_169_cc.tif 	 image_169_cc.png 	 0.9359 	 93.5897 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_256_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_256_cc.png
# 85 :	 image_256_cc.tif 	 image_256_cc.png 	 0.6840 	 68.3970 	 60+ to 70%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_251_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_251_cc.png
# 86 :	 image_251_cc.tif 	 image_251_cc.png 	 0.9507 	 95.0652 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/im

# 169 :	 image_530_cc.tif 	 image_530_cc.png 	 0.9035 	 90.3507 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_130_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_130_cc.png
# 170 :	 image_130_cc.tif 	 image_130_cc.png 	 0.8934 	 89.3392 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_55_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_55_cc.png
# 171 :	 image_55_cc.tif 	 image_55_cc.png 	 0.9631 	 96.3150 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/ima

# 255 :	 image_279_cc.tif 	 image_279_cc.png 	 0.9034 	 90.3407 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_446_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_446_cc.png
# 256 :	 image_446_cc.tif 	 image_446_cc.png 	 0.9508 	 95.0808 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_229_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_229_cc.png
# 257 :	 image_229_cc.tif 	 image_229_cc.png 	 0.6659 	 66.5883 	 60+ to 70%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc

# 340 :	 image_622_cc.tif 	 image_622_cc.png 	 0.7411 	 74.1075 	 70+ to 80%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_308_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_308_cc.png
# 341 :	 image_308_cc.tif 	 image_308_cc.png 	 0.8406 	 84.0555 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_280_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_280_cc.png
# 342 :	 image_280_cc.tif 	 image_280_cc.png 	 0.8526 	 85.2608 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_97_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_97_cc.png
# 424 :	 image_97_cc.tif 	 image_97_cc.png 	 0.9536 	 95.3551 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_384_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_384_cc.png
# 425 :	 image_384_cc.tif 	 image_384_cc.png 	 0.9668 	 96.6804 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/Image_175_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-da

# 509 :	 image_490_cc.tif 	 image_490_cc.png 	 0.9581 	 95.8064 	 95+ to 100%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_333_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_333_cc.png
# 510 :	 image_333_cc.tif 	 image_333_cc.png 	 0.9382 	 93.8241 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_143_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_143_cc.png
# 511 :	 image_143_cc.tif 	 image_143_cc.png 	 0.8539 	 85.3931 	 80+ to 90%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc

# 596 :	 image_571_cc.tif 	 image_571_cc.png 	 0.0044 	 0.4373 	  <= 20%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_289_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_289_cc.png
# 597 :	 image_289_cc.tif 	 image_289_cc.png 	 0.0000 	 0.0000 	  <= 20%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_538_cc.tif
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/resnet-50-out-cc/image_538_cc.png
# 598 :	 image_538_cc.tif 	 image_538_cc.png 	 0.9162 	 91.6236 	 90+ to 95%
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
/home/ec2-user/SageMaker/data/footprint/mohan-func-test-run-data/label_626_cc/image_50